In [1]:
import pandas as pd


devices=pd.read_csv('devices.csv') 
users=pd.read_csv('users.csv')
advertisements=pd.read_csv('advertisement.csv')

,user_id,timestamp,device_id,clicked
0,U790620,2022-01-01 00:23:33 Saturday,M003,False
1,U584867,2022-01-01 01:30:07 Saturday,M001,False
2,U128681,2022-01-01 01:30:14 Saturday,M002,True
3,U694898,2022-01-01 01:31:55 Saturday,M003,False
4,U456823,2022-01-01 03:18:25 Saturday,M001,False


Counting the unique number of `user_id`s in `users` and  `advertisements`.

In [4]:
num_unique_users=users['user_id'].nunique()
num_unique_advertisements=advertisements['user_id'].nunique()
print(f'number of unique users : {num_unique_users}')
print(f'number of unique advertisements : {num_unique_advertisements}')

number of unique users : 15122
number of unique advertisements : 14602


In [5]:
diff=num_unique_users - num_unique_advertisements
print(f'difference : {diff}')

difference : 520


## merging two dataset to get overall

In [6]:
users_ads=pd.merge(left=users,
                   right=advertisements,
                   left_on=['user_id','timestamp'],
                   right_on=['user_id','timestamp'],
                   how='inner'
                  )
users_ads.head()

,user_id,timestamp,device_id,clicked,ad_source,ad_version
0,U790620,2022-01-01 00:23:33 Saturday,M003,False,Twitter,A
1,U584867,2022-01-01 01:30:07 Saturday,M001,False,Google,B
2,U128681,2022-01-01 01:30:14 Saturday,M002,True,TikTok,B
3,U694898,2022-01-01 01:31:55 Saturday,M003,False,TikTok,A
4,U456823,2022-01-01 03:18:25 Saturday,M001,False,Google,A


## Calculating The  Click-Through Rate

In [12]:
ad_view_count = users_ads.groupby('ad_version').agg({'user_id':'count'})
ad_view_count.columns = ['num_views']
ad_view_count=ad_view_count.reset_index()
ad_view_count

,ad_version,num_views
0,A,7154
1,B,7270


In [13]:
diff=ad_view_count['num_views'][1] - ad_view_count['num_views'][0]
diff

116

 **ad_version B was viewed  more than A  by users in users_ads.**

## Getting Unique `ad_view_count` Counts


In [14]:
ad_view_count = users_ads.groupby('ad_version').agg({'user_id':'nunique'})
ad_view_count.columns = ['num_views']
ad_view_count=ad_view_count.reset_index()
ad_view_count

,ad_version,num_views
0,A,7125
1,B,7232


In [15]:
diff=ad_view_count['num_views'][1] - ad_view_count['num_views'][0]
diff

107

## Percentage of users who clicked on each Ad

In [20]:
ad_ctr_pct = users_ads.groupby('ad_version').agg({'clicked':'mean'}) * 100
ad_ctr_pct.columns = ['click_rate_percentage']
ad_ctr_pct=ad_ctr_pct.reset_index()
ad_ctr_pct['click_rate_percentage']=round(ad_ctr_pct['click_rate_percentage'],2)
ad_ctr_pct

,ad_version,click_rate_percentage
0,A,12.41
1,B,19.42


## Ad Performances by Social Media Platform

In [32]:
social = users_ads.groupby(
    ['ad_source', 'ad_version']).agg(
    {'clicked':'mean'})
social.columns = ['ctr']
social=social.reset_index()
social
pivot_table=pd.pivot_table(
    users_ads,
    index='ad_source',
    columns='ad_version',
    values='clicked',
    aggfunc='mean' ## to get the percentage 
)
pivot_table=pivot_table.reset_index()
social['ctr']=round(social['ctr'],2) * 100
social.sort_values(by='ctr',ascending=False)

,ad_source,ad_version,ctr
1,Google,B,20.0
5,TikTok,B,20.0
3,Meta,B,19.0
7,Twitter,B,18.0
0,Google,A,13.0
2,Meta,A,13.0
4,TikTok,A,12.0
6,Twitter,A,12.0


## Ad Performances by Tech Device

In [34]:
devices.head()

,device_id,device_type,brand
0,M001,Mobile,Apple
1,M002,Mobile,Samsung
2,M003,Mobile,Google
3,M004,Mobile,Huawei
4,M005,Mobile,Xiaomi


**combining each user's ad information with their device information**

In [35]:
users_devices = pd.merge(left=users_ads, 
                         right=devices, 
                         left_on='device_id', 
                         right_on='device_id', 
                         how='left')
users_devices.head()

,user_id,timestamp,device_id,clicked,ad_source,ad_version,device_type,brand
0,U790620,2022-01-01 00:23:33 Saturday,M003,False,Twitter,A,Mobile,Google
1,U584867,2022-01-01 01:30:07 Saturday,M001,False,Google,B,Mobile,Apple
2,U128681,2022-01-01 01:30:14 Saturday,M002,True,TikTok,B,Mobile,Samsung
3,U694898,2022-01-01 01:31:55 Saturday,M003,False,TikTok,A,Mobile,Google
4,U456823,2022-01-01 03:18:25 Saturday,M001,False,Google,A,Mobile,Apple


**the percentage of users who clicked on the advertisement based on their device_type and ad_version they viewed.**

In [36]:
ad_devices = pd.pivot_table(users_devices,
                           index = 'device_type',
                           columns = 'ad_version',
                           values = 'clicked',
                           aggfunc = 'mean')

ad_devices

ad_version,A,B
device_type,,
Mobile,0.121057,0.215877
PC,0.121311,0.182425
Tablet,0.128315,0.171348


## Weekday and Weekend Performance by Device Type

In [40]:
users_devices['day_of_week'] = users_devices['timestamp'].str.split(' ', expand=True)[2]

users_day = users_devices.groupby(
    ['day_of_week', 'ad_version']).agg(
    {'user_id':'count'})

users_day.columns=['num_views']
users_day=users_day.reset_index()
users_day.sort_values(by='num_views',ascending=False)

,day_of_week,ad_version,num_views
11,Tuesday,B,1068
0,Friday,A,1061
1,Friday,B,1061
7,Sunday,B,1057
13,Wednesday,B,1054
12,Wednesday,A,1046
4,Saturday,A,1045
3,Monday,B,1037
8,Thursday,A,1018
2,Monday,A,1014


splitting

**filtering for users who saw either ad on weekend or weekdays**

In [41]:
is_weekend = (users_devices['day_of_week'] == 'Saturday') | (users_devices['day_of_week'] == 'Sunday')
weekends = users_devices[is_weekend]
weekdays = users_devices[~is_weekend]

weekdays.head()

,user_id,timestamp,device_id,clicked,ad_source,ad_version,device_type,brand,day_of_week
72,U532019,2022-01-03 00:01:43 Monday,M004,False,Meta,B,Mobile,Huawei,Monday
73,U204646,2022-01-03 00:04:32 Monday,M001,False,Google,A,Mobile,Apple,Monday
74,U484678,2022-01-03 00:10:18 Monday,M003,False,Meta,A,Mobile,Google,Monday
75,U676966,2022-01-03 00:39:32 Monday,M002,True,Google,B,Mobile,Samsung,Monday
76,U145580,2022-01-03 00:45:24 Monday,M001,False,Google,A,Mobile,Apple,Monday


Applying agg

getting the total number of views on weekdays
getting the percent of clicks on weekdays

In [47]:
weekday_ctr = weekdays.groupby(
    ['device_type','ad_version']).agg(
    {'user_id':'count',
     'clicked':'mean'})

weekday_ctr.columns = ['weekday_views','weekday_ctr']

weekday_ctr = weekday_ctr.reset_index()

weekday_ctr['weekday_ctr']=round(weekday_ctr['weekday_ctr'],2)*100
weekday_ctr.sort_values(by=['weekday_ctr'],ascending=False)


,device_type,ad_version,weekday_views,weekday_ctr
1,Mobile,B,2594,21.0
3,PC,B,1285,19.0
5,Tablet,B,1012,16.0
4,Tablet,A,1009,14.0
0,Mobile,A,2517,13.0
2,PC,A,1309,13.0


for weekends

In [50]:
weekend_ctr = weekends.groupby(
    ['device_type','ad_version']).agg(
    {'user_id':'count',
     'clicked':'mean'})

weekend_ctr.columns = ['weekend_views','weekend_ctr']

weekend_ctr = weekend_ctr.reset_index()

weekend_ctr['weekend_ctr']=round(weekend_ctr['weekend_ctr'],2)*100
weekend_ctr.sort_values(by=['weekend_ctr'],ascending=False)

,device_type,ad_version,weekend_views,weekend_ctr
1,Mobile,B,996,22.0
5,Tablet,B,412,19.0
3,PC,B,513,17.0
0,Mobile,A,1002,11.0
2,PC,A,521,10.0
4,Tablet,A,386,9.0
